## Importing libraries

In [1]:
import pandas as pd
import numpy as np

# For method 1
from random import randint
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

# For method 2
from numpy import dot
from numpy.linalg import norm 

# For method 3
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split


# Generating data

In [2]:
def generate_data(n_books, n_genres, n_authors, n_publishers, n_readers, dataset_size):
    '''
    This function will generate a dataset with features associated to
    book data set. The dataset will have the following columns : 
        - book_id (String) : Unique identified for the book
        - book_rating (Integer) : A value between 0 and 10
        - reader_id (String) : Unique identifier for the user
        - book_genre (Integer) : An integer representing a genre for the book, 
                                 value is between 1 and 15, indicating that 
                                 there are 15 unique genres. Each book can only
                                 have 1 genre
        - author_id (String) : Unique identifier for the author of the book
        - num_pages (Integer) : Random value between 70 and 500
        - publisher_id (String) : A unique identifier for the publisher of the book
        - publish_year (Integer) : The year of book publishing
        - book_price (Integer) : The sale price of the book
        - text_lang (Integer) : The language of the book - returns an integer which 
                                is mapped to some language
        
    params:
        n_books (Integer) : The number of books you want the dataset to have
        n_genres (Integer) : Number of genres to be chosen from
        n_authors (Integer) : Number of authors to be generated
        n_publishers (Integer) : Number of publishers for the dataset
        n_readers (Integer) : Number of readers for the dataset
        dataset_size (Integer) : The number of rows to be generated 
        
    example:
        data = generate_data()
    '''
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d

In [3]:
d = generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, dataset_size = 100000)
d.to_csv('./data/data.csv', index = False)

# Method 1. Collaborative Filtering System

**Intuition**

Collaborative filtering is the process of predicting the interests of a user by identifying preferences and information from many users. This is done by filtering data for information or patterns using techniques involving collaboration among multiple agents, data sources, etc. The underlying intuition behind collaborative filtering is that if user A and B have similar taste in a product, then A and B are likely to have similar taste in other products as well. 

There are two common types of approaches in collaborative filtering, memory based and model based approach.

**Memory based approaches** — also often referred to as neighbourhood collaborative filtering. Essentially, ratings of user-item combinations are predicted on the basis of their neighbourhoods. This can be further split into user based collaborative filtering and item based collaborative filtering. User based essentially means that likeminded users are going to yield strong and similar recommendations. Item based collaborative filtering recommends items based on the similarity between items calculated using user ratings of those items.

**Model based approaches** — are predictive models using machine learning. Features associated to the dataset are parameterized as inputs of the model to try to solve an optimization related problem. Model based approaches include using things like decision trees, rule based approaches, latent factor models etc.

**Advantages**

The main advantage to using collaborative filtering models is its simplicity to implement and the high level coverage they provide. It is also beneficial because it captures subtle characteristics (very true for latent factor models) and does not require understanding of the item content.

**Disadvantages**

The main disadvantage to this model is that it’s not friendly for recommending new items, this is because there has been no user/item interaction with it. This is referred to as the cold start problem. Memory based algorithms are known to perform poorly on highly sparse datasets.

**Examples**

Some examples of collaborative filtering algorithms :
- YouTube content recommendation to users — recommending you videos based on other users who have subscribed / watched similar videos as yourself.
- CourseEra course recommendation — recommending you courses based on other individuals who have finished existing courses you’ve finished.

**Implementation**

- Import data from generate_data function (function provided above)
- Generate a pivot table with readers on the index and books on the column and values being the ratings
- Calculate similarity between items and users using svds
- Generate item recommendations based on user_id

## Functions

In [4]:
def normalize_pred_ratings(pred_ratings):
    '''
    This function will normalize the input pred_ratings
    
    params:
        pred_ratings (List -> List) : The prediction ratings 
    '''
    return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

In [5]:
def generate_prediction_df(mat, pt_df, n_factors):
    '''
    This function will calculate the single value decomposition of the input matrix
    given n_factors. It will then generate and normalize the user rating predictions.
    
    params:
        mat (CSR Matrix) : scipy csr matrix corresponding to the pivot table (pt_df)
        pt_df (DataFrame) : pandas dataframe which is a pivot table
        n_factors (Integer) : Number of singular values and vectors to compute. 
                              Must be 1 <= n_factors < min(mat.shape). 
    '''
    
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
        
    # matrix factorization
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # calculate pred ratings
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize_pred_ratings(pred_ratings)
    
    # convert to df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df

In [6]:
def recommend_items(pred_df, usr_id, n_recs):
    '''
    Given a usr_id and pred_df this function will recommend
    items to the user.
    
    params:
        pred_df (DataFrame) : generated from `generate_prediction_df` function
        usr_id (Integer) : The user you wish to get item recommendations for
        n_recs (Integer) : The number of recommendations you want for this user
    '''
    
    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    return rec_df

## Implementation

### Reading data

In [7]:
PATH = './data/data.csv'

In [8]:
data = pd.read_csv(PATH)
print(data.shape)


(100000, 10)


In [9]:
data.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,837,355,7,19273,588,7,30,2006,98,5
1,1834,33,7,907,547,3,25,2016,85,7
2,2649,42,3,25690,292,5,15,2007,135,3
3,1488,178,7,14467,213,2,22,2020,52,3
4,2708,53,6,14950,252,9,7,2004,93,1


### Generating a pivot table with readers on the index and books on the column and values being the ratings

In [10]:
# Creating a copy
df = data.copy()

# Pivot table
pt_df = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

pt_df

book_id,1,2,3,4,5,6,7,8,9,10,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
reader_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Converting to a csr matrix

In [11]:
mat = pt_df.values
mat = csr_matrix(mat)
mat

<28958x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 99934 stored elements in Compressed Sparse Row format>

In [12]:
pred_df = generate_prediction_df(mat, pt_df, 10)
pred_df

,1,2,3,4,5,6,7,8,9,10,...,29991,29992,29993,29994,29995,29996,29997,29998,29999,30000
book_id,,,,,,,,,,,,,,,,,,,,,
1,0.106828,0.106771,0.106741,0.106246,0.106586,0.106708,0.106627,0.106307,0.106380,0.106380,...,0.106169,0.106453,0.106731,0.106725,0.106212,0.106673,0.108775,0.107124,0.106606,0.107379
2,0.106406,0.106032,0.106462,0.106202,0.106321,0.106407,0.106558,0.106215,0.106278,0.106126,...,0.106103,0.106316,0.106902,0.106163,0.106239,0.106344,0.107222,0.106589,0.106706,0.106193
3,0.106801,0.106703,0.106753,0.106275,0.106621,0.106770,0.106788,0.106349,0.106397,0.106251,...,0.106153,0.106498,0.107124,0.106686,0.106233,0.106589,0.108982,0.107014,0.106686,0.107317
4,0.106934,0.107001,0.106106,0.106365,0.106433,0.106840,0.107330,0.106493,0.106684,0.106244,...,0.106350,0.106887,0.107751,0.107546,0.106361,0.106569,0.110497,0.106875,0.106700,0.108484
5,0.106314,0.106342,0.106903,0.106111,0.106028,0.106023,0.106324,0.106085,0.106267,0.106150,...,0.106181,0.106104,0.106101,0.106566,0.106186,0.106137,0.105957,0.106201,0.106082,0.105946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.106655,0.106668,0.106086,0.106199,0.106396,0.106580,0.106864,0.106329,0.106528,0.106164,...,0.106193,0.106366,0.106465,0.106349,0.106200,0.106571,0.110141,0.106748,0.106370,0.106793
2997,0.106681,0.106523,0.106342,0.106345,0.106434,0.106628,0.106914,0.106137,0.106326,0.106610,...,0.106159,0.107100,0.106824,0.106557,0.106414,0.106750,0.106423,0.107283,0.106531,0.107025
2998,0.107021,0.106912,0.106367,0.106260,0.106539,0.106717,0.107317,0.106430,0.106761,0.106201,...,0.106237,0.106504,0.106480,0.106363,0.106262,0.106832,0.112066,0.107026,0.106477,0.106835


### Generating recommendations

In [13]:
recommend_items(pred_df, usr_id=5, n_recs=5)

,book_id,sim
0,1059,0.111791
1,214,0.109573
2,1089,0.107319
3,2196,0.107284
4,2202,0.107241


# Method 2. Content Based Systems

**Intuition**

Content based systems generates recommendations based on the users preferences and profile. They try to match users to items which they’ve liked previously. The level of similarity between items is generally established based on attributes of items liked by the user. Unlike most collaborative filtering models which leverage ratings between target user and other users, content based models focus on the ratings provided by the target user themselves. In essence, the content based approach leverages different sources of data to generate recommendations.

The simplest forms of content based systems require the following sources of data (these requirements can increase based on the complexity of the system you’re trying to build):

1. Item level data source — you need a strong source of data associated to the attributes of the item. For our scenario, we have things like book price, num_pages, published_year, etc. The more information you know regarding the item, the more beneficial it will be for your system.
2. User level data source — you need some sort of user feedback based on the item you’re providing recommendations for. This level of feedback can be either implicit or explicit. In our sample data, we’re working with user ratings of books they’ve read. The more user feedback you can track, the more beneficial it will be for your system.

**Advantages**

Content based models are most advantageous for recommending items when there is an insufficient amount of rating data available. This is because other items with similar attributes might have been rated by the user. Hence, a model should be able to leverage the ratings along with the item attributes to generate recommendations even when there isn’t a lot of data.

**Disadvantages**

There are two main disadvantages of content based systems.
1. The recommendations provided are “obvious” based on the items / content the user has consumed. This is a disadvantage because if the user has never interacted with a particular type of item, that item will never be recommended to the user. For example, if you’ve never read mystery books, then through this approach, you will never be recommended mystery books. This is because the model is user specific and doesn’t leverage knowledge from similar users. This reduces the diversity of the recommendations, this is a negative outcome for many businesses.
2. They’re ineffective for providing recommendations for new users. When building a model you require a history of explicit / implicit user level data for the items. It’s generally important to have a large dataset of ratings available to make robust predictions without overfitting.

**Examples**

Some examples of content based systems are :
- Amazon product feed (you’re being recommended products similar to what you’ve previously purchased)
- Spotify music recommendations

There are many excellent content based systems which are built algorithmically without the dependency on a model based approach. For example companies like Hacker Rank and Reddit have been known to previously used algorithmic approaches to recommend new posts on their platform to users. The key to building an algorithmic approach to content based recommenders lies in defining a set of rules for your business which can be used to rank items. In the case of Reddit, their recommendations are bounded by time of post, number of likes, number of dislikes, number of comments, etc. This can be factored into a formula to generate a score for a post, a high score would yield a high recommendation and vice versa.

**Implementation**

- Import data from generate_data function (function provided above)
- Normalize book_price, book_ratings, num_pages
- One hot encode publish_year, book_genre, text_lang
- Given a book_id input, calculate the cosine similarity and return top n books similar to the input

## Functions

In [14]:
def normalize_input_data(data):
    '''
    This function will normalize the input data to be between 0 and 1
    
    params:
        data (List) : The list of values you want to normalize
    
    returns:
        The input data normalized between 0 and 1
    '''
    min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    return [x/max_val for x in data]

In [15]:
def ohe(df, enc_col):
    '''
    This function will one hot encode the specified column and add it back
    onto the input dataframe
    
    params:
        df (DataFrame) : The dataframe you wish for the results to be appended to
        enc_col (String) : The column you want to OHE
    
    returns:
        The OHE columns added onto the input dataframe
    '''
    
    ohe_df = pd.get_dummies(df[enc_col])
    ohe_df.reset_index(drop = True, inplace = True)
    return pd.concat([df, ohe_df], axis = 1)

In [16]:
class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        '''
        This function will calculate the cosine similarity between two vectors
        '''
        return sum(dot(v1,v2)/(norm(v1)*norm(v2)))
    
    def recommend(self, book_id, n_rec):
        """
        df (dataframe): The dataframe
        song_id (string): Representing the song name
        n_rec (int): amount of rec user wants
        """
        
        # calculate similarity of input book_id vector w.r.t all other vectors
        inputVec = self.df.loc[book_id].values
        self.df['similarity']= self.df.apply(lambda x: self.cosine_sim(inputVec, x.values), axis=1)

        # returns top n user specified books
        return self.df.nlargest(columns='similarity',n=n_rec)

## Implementation

### Reading data

In [17]:
PATH = './data/data.csv'

In [18]:
data = pd.read_csv(PATH)
print(data.shape)


(100000, 10)


In [19]:
data.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,837,355,7,19273,588,7,30,2006,98,5
1,1834,33,7,907,547,3,25,2016,85,7
2,2649,42,3,25690,292,5,15,2007,135,3
3,1488,178,7,14467,213,2,22,2020,52,3
4,2708,53,6,14950,252,9,7,2004,93,1


### Transformations

In [20]:
# Creating a copy
df = data.copy()

# Normalizing the num_pages, ratings, price columns
df['num_pages_norm'] = normalize_input_data(df['num_pages'].values)
df['book_rating_norm'] = normalize_input_data(df['book_rating'].values)
df['book_price_norm'] = normalize_input_data(df['book_price'].values)

# OHE on publish_year and genre
df = ohe(df = df, enc_col = 'publish_year')
df = ohe(df = df, enc_col = 'book_genre')
df = ohe(df = df, enc_col = 'text_lang')

# Drop redundant columns
cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
df.drop(columns = cols, inplace = True)
df.set_index('book_id', inplace = True)

df.head()

,author_id,reader_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,...,8,9,10,1,2,3,4,5,6,7
book_id,,,,,,,,,,,,,,,,,,,,,
837,355,19273,30,0.840000,0.7,0.490,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1834,33,907,25,0.781429,0.3,0.425,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2649,42,25690,15,0.417143,0.5,0.675,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1488,178,14467,22,0.304286,0.2,0.260,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2708,53,14950,7,0.360000,0.9,0.465,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


### Run on a sample as an example

In [21]:
t = df.copy()
cbr = CBRecommend(df = t)
cbr

In [22]:
cbr.recommend(book_id = 1042, n_rec = 5)[['author_id', 'book_rating_norm', 'similarity']].sort_values(by=['book_rating_norm'], ascending = False)

,author_id,book_rating_norm,similarity
book_id,,,
80,410,1.0,5.748098
316,366,0.7,5.748098
1538,379,0.5,5.748098
1170,288,0.5,5.748098
2239,330,0.3,5.748098


# Method 3. Hybrid Recommendation System

**Intuition**

Various methods of recommendations systems have their own benefits and flaws. Often, many of these methods may seem restrictive when used in isolation, especially when multiple sources of data is available for the problem. Hybrid recommender systems are ones designed to use different available data sources to generate robust inferences.

Hybrid recommendation systems have two predominant designs, parallel and sequential. The parallel design provides the input to multiple recommendation systems, each of those recommendations are combined to generate one output. The sequential design provides the input parameters to a single recommendation engine, the output is passed on to the following recommender in a sequence.

**Advantages**

Hybrid systems combine different models to combat the disadvantages of one model with another. This overall reduces the weaknesses of using individual models and aids in generating more robust recommendations. This yields more robust and personalized recommendations for users.

**Disadvantages**

These types of models generally have high computational complexity and require a large database of ratings and other attributes to keep up to date. Without up to date metrics (user engagement, ratings, etc.) it makes it difficult to retrain and provide new recommendations with updated items and ratings from various users.

**Examples**

Netflix is a company which uses a hybrid recommendation system, they generate recommendations to users based on the watch and search style of similar users (collaborative filtering) in conjunction with movies which share similar characteristics who’ve been rated by users (content based).

**Implementation**

- Import data from generate_data function (function provided above)
- Use a content-based model (cosine_similarity) to compute the 50 most similar books
- Compute the predicted ratings that the user might give these 50 books using a collaborative filtering model (SVD)
- Return the top n books with the highest predicted rating

## Functions

In [23]:
def hybrid(reader_id, book_id, n_recs, df, cosine_sim, svd_model):
    '''
    This function represents a hybrid recommendation system, it will have the following flow:
        1. Use a content-based model (cosine_similarity) to compute the 50 most similar books
        2. Compute the predicted ratings that the user might give these 50 books using a collaborative filtering model (SVD)
        3. Return the top n books with the highest predicted rating
        
    params:
        reader_id (Integer) : The reader_id 
        book_id (Integer) : The book_id 
        n_recs (Integer) : The number of recommendations you want
        df (DataFrame) : Original dataframe with all book information 
        cosine_sim (DataFrame) : The cosine similarity dataframe
        svd_model (Model) : SVD model
    '''
    
    # sort similarity values in decreasing order and take top 50 results
    similarity = list(enumerate(cosine_sim[int(book_id)]))
    similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
    similarity = similarity[1:50]
    
    # get book metadata
    book_idx = [i[0] for i in similarity]
    books = df.iloc[book_idx][['book_id', 'book_rating', 'num_pages', 'publish_year', 'book_price', 'reader_id']]
    
    # predict using the svd_model
    books['est'] = books.apply(lambda x: svd_model.predict(reader_id, x['book_id'], x['book_rating']).est, axis = 1)
    
    # sort predictions in decreasing order and return top n_recs
    books = books.sort_values('est', ascending=False)
    return books.head(n_recs)

## Implementation

### Reading data

In [24]:
PATH = './data/data.csv'

In [25]:
data = pd.read_csv(PATH)
print(data.shape)


(100000, 10)


In [26]:
data.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,837,355,7,19273,588,7,30,2006,98,5
1,1834,33,7,907,547,3,25,2016,85,7
2,2649,42,3,25690,292,5,15,2007,135,3
3,1488,178,7,14467,213,2,22,2020,52,3
4,2708,53,6,14950,252,9,7,2004,93,1


### Content based

In [27]:
# Creating a copy
df = data.copy()

# Pivot table
rmat = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

### Compute the cosine similarity matrix 

In [28]:
cosine_sim = cosine_similarity(rmat, rmat)
cosine_sim = pd.DataFrame(cosine_sim, index=rmat.index, columns=rmat.index)
cosine_sim

reader_id,1,2,3,4,5,6,7,8,9,10,...,29991,29992,29993,29994,29995,29996,29997,29998,29999,30000
reader_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.807501,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.163269,0.0,...,0.0,0.000000,0.0,0.0,0.0,1.000000,0.0,0.195069,0.0,0.0
29997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0,0.000000,0.0,0.0
29998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.195069,0.0,1.000000,0.0,0.0


### Collaborative filtering

In [29]:
reader = Reader()
data = Dataset.load_from_df(df[['reader_id', 'book_id', 'book_rating']], reader)
data

### Modeling

In [30]:
# split data into train test
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

# train model
svd = SVD()
svd.fit(trainset)

In [31]:
# run the trained model against the testset
test_pred = svd.test(testset)

# get RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 2.9336


2.933642996682108

### Generate recommendations

In [32]:
hybrid(reader_id = df['reader_id'].values[0], 
       book_id = df['book_id'].values[0], 
       n_recs = 10, 
       df=df, 
       cosine_sim=cosine_sim, 
       svd_model=svd
      ).sort_values(by=['book_rating'], ascending = False)

,book_id,book_rating,num_pages,publish_year,book_price,reader_id,est
5788,2240,8,245,2004,18,5571,5.0
28400,902,6,582,2003,6,23659,5.0
10480,2224,5,574,2012,83,23709,5.0
17151,918,5,351,2006,162,6165,5.0
27166,1240,5,651,2017,165,16752,5.0
16529,1,4,197,2016,149,15541,5.0
24150,449,2,619,2019,86,27660,5.0
25407,602,1,305,2002,187,8571,5.0
28328,2841,1,520,2011,155,11061,5.0
19730,1236,1,423,2017,56,2358,5.0


In [33]:
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,837,355,7,19273,588,7,30,2006,98,5
1,1834,33,7,907,547,3,25,2016,85,7
2,2649,42,3,25690,292,5,15,2007,135,3
3,1488,178,7,14467,213,2,22,2020,52,3
4,2708,53,6,14950,252,9,7,2004,93,1
